In [1]:
import numpy as np

# Define state 1, 2, 3 ,4 ,5... 19 as normal state with one-hot encoding
# state 0 and state 20 share the same zero feature vectors.

def feature_map(state):
    zero_model = [0]*19
    zero_model[state-1] = 1
    zero_model = np.array(zero_model)
    zero_model.resize((19,1))
    return np.array(zero_model)

# create a hash table to quickly draw features
feature_hash = {0: np.zeros((19,1)),
                20: np.zeros((19,1))}
for state in range(1,20):
    feature_hash[state] = feature_map(state)


In [31]:
history = []
for episode in range(100):
    local = [10]
    state = 10
    while True:
        if np.random.random() > 0.5:
            state += 1
        else:
            state -= 1
        local.append(state)
        if state == 0 or state == 20:
            history.append(local)
            break

In [32]:
# hand pick the hyper parameters
alpha = 0.4
gamma = 0.8
_lambda = 0.9

# set all ones as initialization
w_last_episode = np.ones((19,1))
w_last_round = np.ones((19,1))
w_forwad = {}
def n_step_G(t, h, w, hist):
    if h == len(hist):
        # v(T) == 0; reward == 1
        if hist[-1] == 20:
            return gamma**(h-t-1)
        else:
            return 0
    else:
        # reward == 0; 
        return gamma**(h-t)*(w.T@feature_hash[hist[h-1]])
    
def lambda_G(t,h,w,hist):
    first_term = np.sum([_lambda**(n-1)*n_step_G(t,t+n,w,hist) for n in range(1, h-t)])
    return (1-_lambda)*first_term + _lambda**(h-t-1)*n_step_G(t,h,w,hist)
        
for i,hist in enumerate(history):
    for h in range(1, len(hist)+1):
        print(f'processing episode {i} horizon {h}', end = '\r')
        w_old = w_last_episode
        for t in range(1,h+1):
            w = w_old + alpha*(lambda_G(t-1,h,w_last_round,hist) - w_old.T@feature_hash[hist[t-1]])*feature_hash[hist[t-1]]
            w_old = w
        else:
            w_last_round = w_old
    else:
        w_forwad[i] = w_old
        w_last_episode = w_old


In [34]:
w_online = {}
w_2 = np.ones((19,1))
for episode, hist in enumerate(history):
    z = np.zeros((19,1))
    V_old = 0
    for i, state in enumerate(hist):
        if i == len(hist)-2:
            if hist[i+1] == 20:
                R = 1
            else:
                R = 0
            done = True
        else:
            R = 0
            done = False
        V = w_2.T@feature_hash[state]
        V_prime = w_2.T@feature_hash[hist[i+1]]
        delta = R + gamma*V_prime - V
        z = _lambda*gamma*z + (1-alpha*gamma*_lambda*z.T@feature_hash[state])*feature_hash[state]
        w_2 = w_2 + alpha*(delta + V - V_old)*z - alpha*(V-V_old)*feature_hash[state]
        V_old = V_prime
        if done:
            w_online[episode] = w_2
            break

In [40]:
w_forwad[50]

array([[1.31798012e-08],
       [4.68699804e-08],
       [6.16496987e-07],
       [7.95557010e-06],
       [1.70280678e-05],
       [1.58795234e-05],
       [3.21303608e-05],
       [3.03446508e-05],
       [2.27576764e-05],
       [1.70084391e-05],
       [7.81754987e-05],
       [1.97712954e-03],
       [4.46737307e-03],
       [7.13202019e-03],
       [1.74412455e-02],
       [7.50803351e-02],
       [2.39496217e-01],
       [4.09064097e-01],
       [6.29284125e-01]])

In [41]:
w_online[50]

array([[6.87564707e-09],
       [2.30857761e-08],
       [1.55508184e-07],
       [2.20265592e-06],
       [3.27813400e-05],
       [4.05292711e-05],
       [1.09738951e-04],
       [1.80328232e-04],
       [2.70760775e-04],
       [2.59699617e-04],
       [5.85462425e-04],
       [8.92532966e-03],
       [1.99593578e-02],
       [1.52580817e-02],
       [3.64472068e-02],
       [1.40501042e-01],
       [3.56237306e-01],
       [5.67114740e-01],
       [8.31488236e-01]])